In [ ]:
from LinzhUtil import *
import pandas as pd
import time
from sklearn.preprocessing import LabelEncoder


In [ ]:
index = 30
fileList = getFileList("./busDataset")
full_pd = pd.read_csv("./normal2.csv")
full_pd = full_pd.fillna(method='ffill')
full_pd["Unnamed: 0"] = index
index += 1

In [ ]:
for file in fileList:
    if "err" in file:
        continue
    tmp_pd = pd.read_csv("./busDataset/{}".format(file))
    tmp_pd = tmp_pd.fillna(method='ffill')
    tmp_pd["Unnamed: 0"] = index
    index += 1
    full_pd = pd.concat((tmp_pd, full_pd), axis=0, ignore_index=True)
    # break

In [ ]:
full_pd = full_pd.rename(columns={"Unnamed: 0": "group"})
full_pd = full_pd.dropna(axis=1, how='all')
full_pd = full_pd.loc[:, (full_pd != full_pd.iloc[0]).any()]
# full_pd = full_pd.drop('自定义83', axis=1)
# full_pd = full_pd.drop('自定义82', axis=1)
full_pd = full_pd[full_pd["车速"] != "异常"]
full_pd['车速'] = full_pd['车速'].astype("float64")
full_pd["加速度"] = full_pd["车速"].diff(1)
full_pd["加速度"][0] = 0


In [ ]:
for i, e in enumerate(list(full_pd['车速'])):
    try:
        float(e)
    except Exception as e:
        print("Error at {}: {}".format(i, e))


In [ ]:
if True not in full_pd.isnull():
    full_pd.to_csv("normal_full.csv")
else:
    print("error")

In [ ]:
def time2timestamp(timeStr):
    return int(time.mktime(time.strptime(timeStr, "%y-%m-%d %H:%M:%S")))


def labelEncoder(column):
    encoder = LabelEncoder()
    return encoder.fit_transform(column)


def printNoInstance(ls, type):
    for i, e in enumerate(ls):
        if not isinstance(e, type):
            # print(i, end=' ')
            try:
                type(e)
            except Exception as e:
                print("Error at {}: {}".format(i, e))


In [ ]:
full_data = full_pd
data = full_data[["group", "报文时间", "车速", '总电压', '总电流', 'SOC（%）', 'DC-DC状态', '挡位', '电池单体电压最高值', '电池单体电压最低值', '最高温度值', '最低温度值', '最高报警等级', '加速度']]
data = data.drop(data[data["挡位"] == "停车P挡"].index)
data["报文时间"] = data["报文时间"].map(time2timestamp)
data = data[data["总电压"] != "异常"]
data = data[data['电池单体电压最高值'] != '无效']
data = data[data['电池单体电压最高值'] != '异常']
data = data[data['电池单体电压最低值'] != '无效']
data = data[data['最高温度值'] != '异常']
data = data[data['最低温度值'] != '异常']
data = data.apply(pd.to_numeric, errors='ignore')


In [ ]:
for k, v in dict(data.dtypes).items():
    if v == 'object':
        print(k)
        data[k] = labelEncoder(data[k])

In [ ]:
data = data.astype('float64')
if True not in data.isnull():
    data.to_csv("normal_limit.csv")
else:
    print("error")
